# SimpleAIChat

Here's some fun, hackable examples on how simpleaichat works:

- Creating a [Python coding assistant](examples/notebooks/simpleaichat_coding.ipynb) without any unnecessary accompanying output, allowing 5x faster generation at 1/3rd the cost. ([Colab](https://colab.research.google.com/github/minimaxir/simpleaichat/blob/main/examples/notebooks/simpleaichat_coding.ipynb))
- Allowing simpleaichat to [provide inline tips](examples/notebooks/chatgpt_inline_tips.ipynb) following ChatGPT usage guidelines. ([Colab](https://colab.research.google.com/github/minimaxir/simpleaichat/blob/main/examples/notebooks/chatgpt_inline_tips.ipynb))
- Async interface for [conducting many chats](examples/notebooks/simpleaichat_async.ipynb) in the time it takes to receive one AI message. ([Colab](https://colab.research.google.com/github/minimaxir/simpleaichat/blob/main/examples/notebooks/simpleaichat_async.ipynb))
- Create your own Tabletop RPG (TTRPG) setting and campaign by using [advanced structured data models](examples/notebooks/schema_ttrpg.ipynb). ([Colab](https://colab.research.google.com/github/minimaxir/simpleaichat/blob/main/examples/notebooks/schema_ttrpg.ipynb))

In [2]:
from dotenv import load_dotenv
load_dotenv()
import os

import json
import pprint as pp
from pprint import PrettyPrinter, pprint, pformat
pprint = pp.PrettyPrinter(sort_dicts=False).pprint
pformat = pp.PrettyPrinter(sort_dicts=False).pformat

from typing import Any, Dict, List, Optional, Union
from rich.console import Console
console = Console()

from simpleaichat import AIChat
import inspect

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

def pretty_print(json_object):
    print(json.dumps(json_object, indent=2, sort_keys=False, default=pformat))

def console_print(*objects: Any, title=None, title_color="bold white", sep="\n\n", **kwargs: Any) -> None:
    # obj is first objects
    # obj = objects[0]
    title = f"{objects[0].__class__.__name__}:" if title is None else title # if inspect.isclass(obj) else title
    title = f"[{title_color}]{title}[/{title_color}]" if title_color is not None else title
    console.print(title, *objects, sep="\n\n", **kwargs)

def print_ai(ai: AIChat, default=repr):
    for i, (key, val) in enumerate(zip(ai.sessions.keys(), ai.sessions.values())):
        console_print(val, title=f"{val.__repr_name__()} {i}: {default(key)}")

ai = AIChat(console=False)
print_ai(ai)
ai = AIChat(console=False, id="function")
print_ai(ai)

# console_print(ai)
# console_print(ai, title="AIChat object:", title_color="bold green")
# console_print(ai, title="AIChat object:", title_color="bold white")
# console_print(ai, title="AIChat object:", title_color="bold cyan")
# console.print(f"[bold green]Final AIChat object:[/bold green]", ai, sep="\n\n")
# console_print(ai)

True

ChatGPTSession 0: UUID('185bef0d-c869-4f80-a0bc-1bb0ed4749bc')
ChatGPTSession(
    id=UUID('185bef0d-c869-4f80-a0bc-1bb0ed4749bc'),
    created_at=datetime.datetime(2023, 6, 20, 10, 12, 58, 407313, tzinfo=datetime.timezone.utc),
    auth={'api_key': SecretStr('**********')},
    api_url='https://api.openai.com/v1/chat/completions',
    model='gpt-3.5-turbo',
    system='You are a helpful assistant.',
    params={'temperature': 0.7},
    messages=[],
    input_fields={'role', 'name', 'content'},
    recent_messages=None,
    save_messages=True,
    total_prompt_length=0,
    total_completion_length=0,
    total_length=0,
    title=None
)

ChatGPTSession 0: 'function'
ChatGPTSession(
    id='function',
    created_at=datetime.datetime(2023, 6, 20, 10, 12, 58, 475281, tzinfo=datetime.timezone.utc),
    auth={'api_key': SecretStr('**********')},
    api_url='https://api.openai.com/v1/chat/completions',
    model='gpt-3.5-turbo',
    system='You are a helpful assistant.',
    params={'temperature': 0.7},
    messages=[],
    input_fields={'role', 'name', 'content'},
    recent_messages=None,
    save_messages=True,
    total_prompt_length=0,
    total_completion_length=0,
    total_length=0,
    title=None
)

In [3]:
SOURCE_CODE = {}

def add_source_code(obj):
    obj_name = obj.__name__
    source_code = inspect.getsource(obj)
    SOURCE_CODE[obj_name] = source_code

def get_source_code():
    return SOURCE_CODE

add_source_code(AIChat)
# SOURCE_CODE["AIChat"] = inspect.getsource(AIChat)
get_source_code()

{'AIChat': 'class AIChat(BaseModel):\n    client: Union[Client, AsyncClient]\n    default_session: Optional[ChatSession]\n    sessions: Dict[Union[str, UUID], ChatSession] = {}\n\n    class Config:\n        arbitrary_types_allowed = True\n        json_loads = orjson.loads\n        json_dumps = orjson_dumps\n\n    def __init__(\n        self,\n        character: str = None,\n        character_command: str = None,\n        system: str = None,\n        id: Union[str, UUID] = uuid4(),\n        prime: bool = True,\n        default_session: bool = True,\n        console: bool = True,\n        **kwargs,\n    ):\n\n        client = Client()\n        system_format = self.build_system(character, character_command, system)\n\n        sessions = {}\n        new_default_session = None\n        if default_session:\n            new_session = self.new_session(\n                return_session=True, system=system_format, id=id, **kwargs\n            )\n\n            new_default_session = new_session\n  

## Building AI-based Apps


The trick with working with new chat-based apps that wasn't readily available with earlier iterations of GPT-3 is the addition of the system prompt: a different class of prompt that guides the AI behavior throughout the entire conversation. In fact, the chat demos above are actually using [system prompt tricks](https://github.com/minimaxir/simpleaichat/blob/main/PROMPTS.md#interactive-chat) behind the scenes! OpenAI has also released an official guide for [system prompt best practices](https://platform.openai.com/docs/guides/gpt-best-practices) to building AI apps.

For developers, you can instantiate a programmatic instance of `AIChat` by explicitly specifying a system prompt, or by disabling the console.

In [7]:
ai = AIChat(system="You are a helpful assistant.")
print('## AIChat(system="You are a helpful assistant."):\n')
print(ai)
# pprint(ai.get_session().dict())

ai = AIChat(console=False)  # same as above
print('\n## AIChat(console=False):\n')
print(ai)
# pprint(ai.get_session().dict())

## AIChat(system="You are a helpful assistant."):

{
  "id": "65ce31a3-1930-494d-a2bb-9c98cfbb28f9",
  "created_at": "2023-06-20T07:46:43.997356+00:00",
  "auth": {
    "api_key": "**********"
  },
  "model": "gpt-3.5-turbo",
  "system": "You are a helpful assistant.",
  "params": {
    "temperature": 0.7
  },
  "messages": [],
  "input_fields": [
    "name",
    "content",
    "role"
  ],
  "save_messages": true,
  "total_prompt_length": 0,
  "total_completion_length": 0,
  "total_length": 0
}

## AIChat(console=False):

{
  "id": "65ce31a3-1930-494d-a2bb-9c98cfbb28f9",
  "created_at": "2023-06-20T07:46:44.053755+00:00",
  "auth": {
    "api_key": "**********"
  },
  "model": "gpt-3.5-turbo",
  "system": "You are a helpful assistant.",
  "params": {
    "temperature": 0.7
  },
  "messages": [],
  "input_fields": [
    "name",
    "content",
    "role"
  ],
  "save_messages": true,
  "total_prompt_length": 0,
  "total_completion_length": 0,
  "total_length": 0
}


You can also pass in a `model` parameter, such as `model="gpt-4"` if you have access to GPT-4, or `model="gpt-3.5-turbo-16k"` for a larger-context-window ChatGPT.


```python
ai = AIChat(
    console=False,
    model="gpt-3.5-turbo-0613",
    params={"temperature": 0.0},
)
```


You can then feed the new `ai` class with user input, and it will return and save the response from ChatGPT:

In [7]:
response = ai("What is the capital of California?")
print(response)
print(ai)

The capital of California is Sacramento.


Alternatively, you can stream responses by token with a generator if the text generation itself is too slow:

In [13]:
from rich.console import Console
console = Console()

for chunk in ai.stream("What is the capital of California?", params={"max_tokens": 5}):
    response_td = chunk  # dict contains "delta" for the new token and "response"
    # response_td = response_td["response"]  # dict contains "delta" for the new token and "response"
    # print(f'"delta": "{response_td["delta"]}", "response": {response_td["response"]}')
    delta = response_td["delta"]
    response = response_td["response"].replace(delta, f"[{delta}]")
    print(response)
print(ai)

[The]
The[ capital]
The capital[ of]
The capital of[ California]
The capital of California[ is]
{
  "id": "9454965b-52bc-4721-9373-ed0624bf7861",
  "created_at": "2023-06-20T02:22:50.412686+00:00",
  "auth": {
    "api_key": "**********"
  },
  "model": "gpt-3.5-turbo",
  "system": "You are a helpful assistant.",
  "params": {
    "temperature": 0.7
  },
  "messages": [
    {
      "role": "user",
      "content": "What is the capital of California?",
      "received_at": "2023-06-20T02:22:50.413529+00:00"
    },
    {
      "role": "assistant",
      "content": "The capital of California is",
      "received_at": "2023-06-20T02:22:51.418384+00:00"
    }
  ],
  "input_fields": [
    "content",
    "role",
    "name"
  ],
  "save_messages": true,
  "total_prompt_length": 0,
  "total_completion_length": 0,
  "total_length": 0
}


Further calls to the ai object will continue the chat, automatically incorporating previous information from the conversation.

In [20]:
ai = AIChat(console=False, model="gpt-3.5-turbo-0301")

response = ai("What is the capital of California?")
print(response)
# print(ai)
response = ai("When was it founded?")
print(response)
print(ai)
# ai.default_session.messages

The capital of California is Sacramento.
Sacramento was founded on February 27, 1850. It was named after the Sacramento River, which runs through the city.
{
  "id": "183d84c2-77cf-461d-a14c-9ed66d4a5050",
  "created_at": "2023-06-20T02:31:58.325404+00:00",
  "auth": {
    "api_key": "**********"
  },
  "model": "gpt-3.5-turbo-0301",
  "system": "You are a helpful assistant.",
  "params": {
    "temperature": 0.7
  },
  "messages": [
    {
      "role": "user",
      "content": "What is the capital of California?",
      "received_at": "2023-06-20T02:31:58.329111+00:00"
    },
    {
      "role": "assistant",
      "content": "The capital of California is Sacramento.",
      "received_at": "2023-06-20T02:31:59.247147+00:00",
      "finish_reason": "stop",
      "prompt_length": 26,
      "completion_length": 7,
      "total_length": 33
    },
    {
      "role": "user",
      "content": "When was it founded?",
      "received_at": "2023-06-20T02:31:59.248271+00:00"
    },
    {
      "

You can also save chat sessions (as CSV or JSON) and load them later. The API key is not saved so you will have to provide that when loading.

In [28]:
# CSV, will only save messages
ai.save_session("chat_session.csv", format="csv")  # CSV
ai.load_session("chat_session.csv")
print("\n## CSV\n")
print(ai)

# JSON
ai.save_session("chat_session.json", format="json", minify=True)  # JSON
ai.load_session("chat_session.json")
print("\n## JSON\n")
print(ai)


## CSV

{
  "id": "183d84c2-77cf-461d-a14c-9ed66d4a5050",
  "created_at": "2023-06-20T02:31:58.325404+00:00",
  "auth": {
    "api_key": "**********"
  },
  "model": "gpt-3.5-turbo-0301",
  "system": "You are a helpful assistant.",
  "params": {
    "temperature": 0.7
  },
  "messages": [
    {
      "role": "user",
      "content": "What is the capital of California?",
      "received_at": "2023-06-20T02:31:58.329111+00:00"
    },
    {
      "role": "assistant",
      "content": "The capital of California is Sacramento.",
      "received_at": "2023-06-20T02:31:59.247147+00:00",
      "finish_reason": "stop",
      "prompt_length": 26,
      "completion_length": 7,
      "total_length": 33
    },
    {
      "role": "user",
      "content": "When was it founded?",
      "received_at": "2023-06-20T02:31:59.248271+00:00"
    },
    {
      "role": "assistant",
      "content": "Sacramento was founded on February 27, 1850. It was named after the Sacramento River, which runs through the 

### Functions

A large number of popular venture-capital-funded ChatGPT apps don't actually use the "chat" part of the model. Instead, they just use the system prompt/first user prompt as a form of natural language programming. You can emulate this behavior by passing a new system prompt when generating text, and not saving the resulting messages.

In [4]:
from rich.console import Console
console = Console()

json = '{"title": "An array of integers.", "array": [-1, 0, 1]}'

params = {"temperature": 0.0, "max_tokens": 100}  # a temperature of 0.0 is deterministic

# We namespace the function by `id` so it doesn't affect other chats.
# Settings set during session creation will apply to all generations from the session,
# but you can change them per-generation, as is the case with the `system` prompt here.
system = "Format the user-provided JSON as YAML."
ai = AIChat(console=False, id="function", params=params, save_messages=False)
output = ai(json, id="function", system=system)
console.print(f"[bold magenta]System Prompt:[/bold magenta] {system}", output, sep="\n\n")
# print(output)
print_ai(ai)

System Prompt: Format the user-provided JSON as YAML.

title: "An array of integers."
array:
  - -1
  - 0
  - 1

ChatGPTSession 0: 'function'
ChatGPTSession(
    id='function',
    created_at=datetime.datetime(2023, 6, 20, 10, 13, 49, 142472, tzinfo=datetime.timezone.utc),
    auth={'api_key': SecretStr('**********')},
    api_url='https://api.openai.com/v1/chat/completions',
    model='gpt-3.5-turbo',
    system='You are a helpful assistant.',
    params={'temperature': 0.0, 'max_tokens': 100},
    messages=[],
    input_fields={'role', 'name', 'content'},
    recent_messages=None,
    save_messages=False,
    total_prompt_length=43,
    total_completion_length=24,
    total_length=67,
    title=None
)

In [40]:
# system = "Format the user-provided JSON as YAML."
# console.print(f"[bold magenta]Function:[/bold magenta] {system}", output, sep="\n\n")
# console.print(f"[bold magenta]System Prompt:[/bold magenta] {system}", output, sep="\n\n")
# console.print(f"[bold magenta]Function:[/bold magenta] JSON to YAML", output, sep="\n\n")


In [41]:
from rich.console import Console
console = Console()

functions = [
             "Format the user-provided JSON as YAML.",
             "Write a limerick based on the user-provided JSON.",
             "Translate the user-provided JSON from English to French."
            ]
for i, function in enumerate(functions):
    output = ai(json, id="function", system=function)
    # console.print(f"[bold magenta]Output {i+1}:[/bold magenta]\n{output}")
    console.print(f"[bold magenta]System Prompt:[/bold magenta] {system}", output, sep="\n\n")
    
console.print(f"[bold green]Final AIChat object:[/bold green]", ai, sep="\n\n")

System Prompt: Format the user-provided JSON as YAML.

title: "An array of integers."
array:
  - -1
  - 0
  - 1

System Prompt: Format the user-provided JSON as YAML.

An array of integers so neat,
With values that can't be beat,
From negative to positive one,
It's a range that's quite fun,
This JSON is really quite sweet!

System Prompt: Format the user-provided JSON as YAML.

{"titre": "Un tableau d'entiers.", "tableau": [-1, 0, 1]}

Final AIChat object:
AIChat(
    client=<httpx.Client object at 0x7fc6fb149510>,
    default_session=ChatGPTSession(
        id='function',
        created_at=datetime.datetime(2023, 6, 20, 4, 17, 30, 406260, tzinfo=datetime.timezone.utc),
        auth={'api_key': SecretStr('**********')},
        api_url='https://api.openai.com/v1/chat/completions',
        model='gpt-3.5-turbo',
        system='You are a helpful assistant.',
        params={'temperature': 0.0, 'max_tokens': 100},
        messages=[],
        input_fields={'role', 'content', 'name'},
        recent_messages=None,
        save_messages=False,
        total_prompt_length=178,
        total_completion_length=109,
        total_length=287,
        title=None
    ),
    sessions={
        'function': ChatGPTSession(
            id='function',
            created_at=datetime.datetime(2023, 6, 20, 4, 17, 30, 406260, tzinfo=datetime.timezone.utc),
            auth={'api_key': SecretStr('**********')},
            api_url='https://api.openai.com/v1/chat/completions',
            model='gpt-3.5-turbo',
            system='You are a helpful assistant.',
            params={'temperature': 0.0, 'max_tokens': 100},
            messages=[],
            input_fields={'role', 'content', 'name'},
            recent_messages=None,
            save_messages=False,
            total_prompt_length=178,
            total_completion_length=109,
            total_length=287,
            title=None
        )
    }
)

In [9]:
print_ai(ai)

ChatGPTSession 0: UUID("65ce31a3-1930-494d-a2bb-9c98cfbb28f9")
ChatGPTSession(
    id=UUID('65ce31a3-1930-494d-a2bb-9c98cfbb28f9'),
    created_at=datetime.datetime(2023, 6, 20, 7, 47, 13, 635161, tzinfo=datetime.timezone.utc),
    auth={'api_key': SecretStr('**********')},
    api_url='https://api.openai.com/v1/chat/completions',
    model='gpt-3.5-turbo-0613',
    system='You are a helpful assistant.',
    params={'temperature': 0.0},
    messages=[],
    input_fields={'name', 'content', 'role'},
    recent_messages=None,
    save_messages=False,
    total_prompt_length=94,
    total_completion_length=39,
    total_length=133,
    title=None
)

#### Function Calling

Newer versions of ChatGPT also support "[function calling](https://platform.openai.com/docs/guides/gpt/function-calling)", but the real benefit of that feature is the ability for ChatGPT to support structured input and/or output, which now opens up a wide variety of applications! simpleaichat streamlines the workflow to allow you to just pass an `input_schema` and/or an `output_schema`.

You can construct a schema using a [pydantic](https://docs.pydantic.dev/latest/) BaseModel.


In [8]:
from pydantic import BaseModel, Field

ai = AIChat(
    console=False,
    save_messages=False,  # with schema I/O, messages are never saved
    model="gpt-3.5-turbo-0613",
    params={"temperature": 0.0},
)

class get_event_metadata(BaseModel):
    """Event information"""

    description: str = Field(description="Description of event")
    city: str = Field(description="City where event occured")
    year: int = Field(description="Year when event occured")
    month: str = Field(description="Month when event occured")

# returns a dict, with keys ordered as in the schema
response = ai("First iPhone announcement", output_schema=get_event_metadata)
console_print(response, title="iPhone response:")
console_print(ai, title=f"iPhone AIChat object:")#, title_color="bold green")


iPhone response:
{
    'description': 'The first iPhone was announced by Apple Inc.',
    'city': 'San Francisco',
    'year': 2007,
    'month': 'January'
}

iPhone AIChat object:
AIChat(
    client=<httpx.Client object at 0x7f5024d57550>,
    default_session=ChatGPTSession(
        id=UUID('65ce31a3-1930-494d-a2bb-9c98cfbb28f9'),
        created_at=datetime.datetime(2023, 6, 20, 7, 47, 13, 635161, tzinfo=datetime.timezone.utc),
        auth={'api_key': SecretStr('**********')},
        api_url='https://api.openai.com/v1/chat/completions',
        model='gpt-3.5-turbo-0613',
        system='You are a helpful assistant.',
        params={'temperature': 0.0},
        messages=[],
        input_fields={'name', 'content', 'role'},
        recent_messages=None,
        save_messages=False,
        total_prompt_length=94,
        total_completion_length=39,
        total_length=133,
        title=None
    ),
    sessions={
        UUID('65ce31a3-1930-494d-a2bb-9c98cfbb28f9'): ChatGPTSession(
            id=UUID('65ce31a3-1930-494d-a2bb-9c98cfbb28f9'),
            created_at=datetime.datetime(2023, 6, 20, 7, 47, 13, 635161, tzinfo=datetime.timezone.utc),
            auth={'api_key': SecretStr('**********')},
            api_url='https://api.openai.com/v1/chat/completions',
            model='gpt-3.5-turbo-0613',
            system='You are a helpful assistant.',
            params={'temperature': 0.0},
            messages=[],
            input_fields={'name', 'content', 'role'},
            recent_messages=None,
            save_messages=False,
            total_prompt_length=94,
            total_completion_length=39,
            total_length=133,
            title=None
        )
    }
)

See the [TTRPG Generator Notebook](examples/notebooks/schema_ttrpg.ipynb) for a more elaborate demonstration of schema capabilities.


### Tools

One of the most recent aspects of interacting with ChatGPT is the ability for the model to use "tools." As popularized by [LangChain](https://github.com/hwchase17/langchain), tools allow the model to decide when to use custom functions, which can extend beyond just the chat AI itself, for example retrieving recent information from the internet not present in the chat AI's training data. This workflow is analogous to ChatGPT Plugins.

Parsing the model output to invoke tools typically requires a number of shennanigans, but simpleaichat uses [a neat trick](https://github.com/minimaxir/simpleaichat/blob/main/PROMPTS.md#tools) to make it fast and reliable! Additionally, the specified tools return a `context` for ChatGPT to draw from for its final response, and tools you specify can return a dictionary which you can also populate with arbitrary metadata for debugging and postprocessing. Each generation returns a dictionary with the `response` and the `tool` function used, which can be used to set up workflows akin to [LangChain](https://github.com/hwchase17/langchain)-style Agents, e.g. recursively feed input to the model until it determines it does not need to use any more tools.

You will need to specify functions with docstrings which provide hints for the AI to select them:


In [5]:
from simpleaichat.utils import wikipedia_search, wikipedia_search_lookup

# This uses the Wikipedia Search API.
# Results from it are nondeterministic, your mileage will vary.
def search(query: str) -> Dict[str, Union[str, List[str]]]:
    """Search the internet."""
    wiki_matches: List[str] = wikipedia_search(query, n=3)
    return {"context": ", ".join(wiki_matches), "titles": wiki_matches}

def lookup(query: str) -> str:
    """Lookup more information about a topic."""
    page = wikipedia_search_lookup(query, sentences=3)
    return page


In [6]:
params = {"temperature": 0.0, "max_tokens": 100}
ai = AIChat(params=params, console=False)

prompt = "San Francisco tourist attractions"
response = ai(prompt, tools=[search, lookup])
console_print(response, title=f"Response: \"{prompt}\"")
# console_print(ai.default_session, title=f"AIChat Session: \"{prompt}\"")
# print_ai(ai)

Response: "San Francisco tourist attractions"
{
    'context': "Fisherman's Wharf, San Francisco, Tourist attractions in the United States, Lombard Street (San 
Francisco)",
    'titles': [
        "Fisherman's Wharf, San Francisco",
        'Tourist attractions in the United States',
        'Lombard Street (San Francisco)'
    ],
    'tool': 'search',
    'response': "There are many popular tourist attractions in San Francisco, including Fisherman's Wharf and 
Lombard Street. Fisherman's Wharf is a bustling waterfront area known for its seafood restaurants, souvenir shops, 
and sea lion sightings. Lombard Street, on the other hand, is a famous winding street with eight hairpin turns that
offers stunning views of the city. Both of these attractions are must-sees for anyone visiting San Francisco."
}

In [11]:
console_print(ai.default_session)

ChatGPTSession:
ChatGPTSession(
    id=UUID('185bef0d-c869-4f80-a0bc-1bb0ed4749bc'),
    created_at=datetime.datetime(2023, 6, 20, 10, 15, 25, 872122, tzinfo=datetime.timezone.utc),
    auth={'api_key': SecretStr('**********')},
    api_url='https://api.openai.com/v1/chat/completions',
    model='gpt-3.5-turbo',
    system='You are a helpful assistant.',
    params={'temperature': 0.0, 'max_tokens': 100},
    messages=[
        ChatMessage(
            role='user',
            content='San Francisco tourist attractions',
            name=None,
            function_call=None,
            received_at=datetime.datetime(2023, 6, 20, 10, 15, 30, 732758, tzinfo=datetime.timezone.utc),
            finish_reason=None,
            prompt_length=None,
            completion_length=None,
            total_length=None
        ),
        ChatMessage(
            role='assistant',
            content="There are many popular tourist attractions in San Francisco, including Fisherman's Wharf and 
Lombard Street. Fisherman's Wharf is a bustling waterfront area known for its seafood restaurants, souvenir shops, 
and sea lion sightings. Lombard Street, on the other hand, is a famous winding street with eight hairpin turns that
offers stunning views of the city. Both of these attractions are must-sees for anyone visiting San Francisco.",
            name=None,
            function_call=None,
            received_at=datetime.datetime(2023, 6, 20, 10, 15, 30, 732766, tzinfo=datetime.timezone.utc),
            finish_reason=None,
            prompt_length=None,
            completion_length=None,
            total_length=None
        )
    ],
    input_fields={'role', 'name', 'content'},
    recent_messages=None,
    save_messages=True,
    total_prompt_length=133,
    total_completion_length=87,
    total_length=220,
    title=None
)

In [114]:
prompt="Lombard Street?"
response = ai(prompt, tools=[search, lookup])
console_print(response, title=f"Response: \"{prompt}\"")
# console_print(ai.default_session, title=f"AIChat Session: \"{prompt}\"")

Response: "Lombard Street?"
{
    'context': 'Lombard Street is an east–west street in San Francisco, California that is famous for a steep, 
one-block section with eight hairpin turns. Stretching from The Presidio east to The Embarcadero (with a gap on 
Telegraph Hill), most of the street\'s western segment is a major thoroughfare designated as part of U.S. Route 
101. The famous one-block section, claimed to be "the crookedest street in the world", is located along the eastern
segment in the Russian Hill neighborhood.',
    'tool': 'lookup',
    'response': 'Lombard Street is a famous street in San Francisco, California known for its steep, one-block 
section with eight hairpin turns. It stretches from The Presidio to The Embarcadero, with a gap on Telegraph Hill. 
The western segment of the street is a major thoroughfare designated as part of U.S. Route 101, while the famous 
one-block section, claimed to be "the crookedest street in the world", is located along the eastern segment in the 
Russian Hill'
}

In [115]:
prompt = "Thanks for your help!"
response = ai(prompt, tools=[search, lookup])
console_print(response, title=f"Response: \"{prompt}\"")

Response: "Thanks for your help!"
{
    'response': "You're welcome! If you have any more questions or need further assistance, feel free to ask.",
    'tool': None
}

In [1]:
prompts = [
    "San Francisco tourist attractions",
    "Lombard Street?",
    "Thanks for your help!"
]
console_print(ai.default_session, title=f"AIChat Session: {prompts}")
# print_ai(ai)


NameError: name 'console_print' is not defined

In [29]:
from simpleaichat.utils import wikipedia_search, wikipedia_search_lookup

# This uses the Wikipedia Search API.
# Results from it are nondeterministic, your mileage will vary.
def search(query: str) -> Dict[str, Union[str, List[str]]]:
    """Search the internet."""
    wiki_matches: List[str] = wikipedia_search(query, n=3)
    return {"context": ", ".join(wiki_matches), "titles": wiki_matches}

def lookup(query: str) -> str:
    """Lookup more information about a topic."""
    page = wikipedia_search_lookup(query, sentences=3)
    return page

tools = [search, lookup]

params = {"temperature": 0.0, "max_tokens": 100}
ai = AIChat(params=params, console=False, id="tools")

console_print(ai)


AIChat:
AIChat(
    client=<httpx.Client object at 0x7f6ea0432470>,
    default_session=ChatGPTSession(
        id='tools',
        created_at=datetime.datetime(2023, 6, 20, 10, 47, 16, 52325, tzinfo=datetime.timezone.utc),
        auth={'api_key': SecretStr('**********')},
        api_url='https://api.openai.com/v1/chat/completions',
        model='gpt-3.5-turbo',
        system='You are a helpful assistant.',
        params={'temperature': 0.0, 'max_tokens': 100},
        messages=[],
        input_fields={'role', 'name', 'content'},
        recent_messages=None,
        save_messages=True,
        total_prompt_length=0,
        total_completion_length=0,
        total_length=0,
        title=None
    ),
    sessions={
        'tools': ChatGPTSession(
            id='tools',
            created_at=datetime.datetime(2023, 6, 20, 10, 47, 16, 52325, tzinfo=datetime.timezone.utc),
            auth={'api_key': SecretStr('**********')},
            api_url='https://api.openai.com/v1/chat/completions',
            model='gpt-3.5-turbo',
            system='You are a helpful assistant.',
            params={'temperature': 0.0, 'max_tokens': 100},
            messages=[],
            input_fields={'role', 'name', 'content'},
            recent_messages=None,
            save_messages=True,
            total_prompt_length=0,
            total_completion_length=0,
            total_length=0,
            title=None
        )
    }
)

In [37]:

client = ai.client
sess = ai.default_session
console_print(pformat(client.__dict__))
console_print(sess)

str:

{'_base_url': URL(''),
 '_auth': None,
 '_params': QueryParams(''),
 '_headers': Headers({'accept': '*/*', 'accept-encoding': 'gzip, deflate', 'connection': 'keep-alive', 
'user-agent': 'python-httpx/0.24.1'}),
 '_cookies': <Cookies[]>,
 '_timeout': Timeout(timeout=5.0),
 'follow_redirects': False,
 'max_redirects': 20,
 '_event_hooks': {'request': [], 'response': []},
 '_trust_env': True,
 '_default_encoding': 'utf-8',
 '_state': <ClientState.UNOPENED: 1>,
 '_transport': <httpx.HTTPTransport object at 0x7f6e7fcb9c60>,
 '_mounts': {}}

ChatGPTSession:
ChatGPTSession(
    id='tools',
    created_at=datetime.datetime(2023, 6, 20, 10, 47, 16, 52325, tzinfo=datetime.timezone.utc),
    auth={'api_key': SecretStr('**********')},
    api_url='https://api.openai.com/v1/chat/completions',
    model='gpt-3.5-turbo',
    system='You are a helpful assistant.',
    params={'temperature': 0.0, 'max_tokens': 100},
    messages=[],
    input_fields={'role', 'name', 'content'},
    recent_messages=None,
    save_messages=True,
    total_prompt_length=0,
    total_completion_length=0,
    total_length=0,
    title=None
)

In [38]:
# with AIChat(params=params, console=False, id="tools_test").session(id="tools_test") as sess:
#     console_print(sess, title=f"Session: {sess.id!r}")

In [43]:
sess.params

{'temperature': 0.0, 'max_tokens': 100}

In [58]:
from pydantic import HttpUrl
from httpx import Client, AsyncClient
from typing import List, Dict, Union, Set, Any
import orjson

from simpleaichat.chatgpt import ChatGPTSession
from simpleaichat.models import ChatMessage, ChatSession

tool_prompt = """From the list of tools below:
- Reply ONLY with the number of the tool appropriate in response to the user's last message.
- If no tool is appropriate, ONLY reply with \"0\".

{tools}"""

prompts = [
    "San Francisco tourist attractions",
    "Lombard Street?",
    "Thanks for your help!"
]

prompt: str
sess: ChatGPTSession                # = ai.default_session
tools: List[Any]                    # = [search, lookup]
client: Union[Client, AsyncClient]  # = ai.client
system: str                         # = sess.system
save_messages: bool                 # = sess.save_messages
params: Dict[str, Any]              # = sess.params

prompt = prompts[0]
sess = ai.default_session
tools = [search, lookup]
client = ai.client
system = sess.system
save_messages = sess.save_messages
params = sess.params

# call 1: select tool and populate context
tools_list = "\n".join(f"{i+1}: {f.__doc__}" for i, f in enumerate(tools))
tool_prompt_format = tool_prompt.format(tools=tools_list)
print("\n`tool_prompt_format`:\n")
print(tool_prompt_format)

logit_bias_weight = 100
logit_bias = {str(k): logit_bias_weight for k in range(15, 15 + len(tools) + 1)}
print(f"\n`logit_bias`:\n\n{logit_bias}")


`tool_prompt_format`:

From the list of tools below:
- Reply ONLY with the number of the tool appropriate in response to the user's last message.
- If no tool is appropriate, ONLY reply with "0".

1: Search the internet.
2: Lookup more information about a topic.

`logit_bias`:

{'15': 100, '16': 100, '17': 100}


In [59]:
# call 1: select tool and populate context

tool_idx_gen = sess.gen(
    prompt,
    client=client,
    system=tool_prompt_format,
    save_messages=False,
    params={
        "temperature": 0.0,
        "max_tokens": 1,
        "logit_bias": logit_bias,
    },
)
print(f"\n`tool_idx_gen`:\n\n{tool_idx_gen}")

tool_idx = int(tool_idx_gen)
print(f"\n`tool_idx`:\n\n{tool_idx}")


`tool_idx_gen`:

1

`tool_idx`:

1


In [66]:
# if no tool is selected, do a standard generation instead.
if tool_idx == 0:
    context_dict = {
        "response": sess.gen(
            prompt,
            client=client,
            system=system,
            save_messages=save_messages,
            params=params,
        ),
        "tool": None,
    }
else:
    # Get Tool: select tool
    selected_tool = tools[tool_idx - 1]
    # Call Tool: call tool to get context
    context_dict = selected_tool(prompt)
    if isinstance(context_dict, str):
        context_dict = {"context": context_dict}

    # Tool Name: add tool name to context
    context_dict["tool"] = selected_tool.__name__

# print(f"\n`context_dict`:\n\n{pformat(context_dict)}")
console_print(pformat(context_dict), title=f"context_dict:")

context_dict:

{'context': "Fisherman's Wharf, San Francisco, Tourist attractions in the "
            'United States, Lombard Street (San Francisco)',
 'titles': ["Fisherman's Wharf, San Francisco",
            'Tourist attractions in the United States',
            'Lombard Street (San Francisco)'],
 'tool': 'search'}

In [69]:
# call 2: generate from the context
new_system = f"{system or sess.system}\n\nYou MUST use information from the context in your response."
console_print(new_system, title=f"new_system:", title_color="bold magenta")

new_prompt = f"Context: {context_dict['context']}\n\nUser: {prompt}"
console_print(new_prompt, title=f"new_prompt:", title_color="bold magenta")


new_system:

You are a helpful assistant.

You MUST use information from the context in your response.

new_prompt:

Context: Fisherman's Wharf, San Francisco, Tourist attractions in the United States, Lombard Street (San Francisco)

User: San Francisco tourist attractions

In [70]:

context_dict["response"] = sess.gen(
    new_prompt,
    client=client,
    system=new_system,
    save_messages=False,
    params=params,
)
console_print(pformat(context_dict), title=f"context_dict:")


context_dict:

{'context': "Fisherman's Wharf, San Francisco, Tourist attractions in the "
            'United States, Lombard Street (San Francisco)',
 'titles': ["Fisherman's Wharf, San Francisco",
            'Tourist attractions in the United States',
            'Lombard Street (San Francisco)'],
 'tool': 'search',
 'response': 'There are many popular tourist attractions in San Francisco, '
             "including Fisherman's Wharf and Lombard Street. Fisherman's "
             'Wharf is a bustling waterfront area known for its seafood '
             'restaurants, souvenir shops, and sea lion sightings. Lombard '
             'Street, on the other hand, is a famous winding street with eight '
             'hairpin turns that attract visitors from all over the world. '
             'Both of these attractions are must-sees for anyone visiting San '
             'Francisco.'}

In [71]:

# manually append the nonmodified user message + normal AI response
user_message = ChatMessage(role="user", content=prompt)
assistant_message = ChatMessage(role="assistant", content=context_dict["response"])
sess.add_messages(user_message, assistant_message, save_messages)
console_print(sess, title=f"Session: {sess.id!r}")

Session: 'tools'
ChatGPTSession(
    id='tools',
    created_at=datetime.datetime(2023, 6, 20, 10, 47, 16, 52325, tzinfo=datetime.timezone.utc),
    auth={'api_key': SecretStr('**********')},
    api_url='https://api.openai.com/v1/chat/completions',
    model='gpt-3.5-turbo',
    system='You are a helpful assistant.',
    params={'temperature': 0.0, 'max_tokens': 100},
    messages=[
        ChatMessage(
            role='user',
            content='San Francisco tourist attractions',
            name=None,
            function_call=None,
            received_at=datetime.datetime(2023, 6, 20, 11, 23, 55, 867633, tzinfo=datetime.timezone.utc),
            finish_reason=None,
            prompt_length=None,
            completion_length=None,
            total_length=None
        ),
        ChatMessage(
            role='assistant',
            content="There are many popular tourist attractions in San Francisco, including Fisherman's Wharf and 
Lombard Street. Fisherman's Wharf is a bustling waterfront area known for its seafood restaurants, souvenir shops, 
and sea lion sightings. Lombard Street, on the other hand, is a famous winding street with eight hairpin turns that
attract visitors from all over the world. Both of these attractions are must-sees for anyone visiting San 
Francisco.",
            name=None,
            function_call=None,
            received_at=datetime.datetime(2023, 6, 20, 11, 23, 55, 867793, tzinfo=datetime.timezone.utc),
            finish_reason=None,
            prompt_length=None,
            completion_length=None,
            total_length=None
        )
    ],
    input_fields={'role', 'name', 'content'},
    recent_messages=None,
    save_messages=True,
    total_prompt_length=133,
    total_completion_length=88,
    total_length=221,
    title=None
)

In [ ]:

def gen_with_tools(
    chat_gpt_session: ChatGPTSession,
    prompt: str,
    tools: List[Any],
    client: Union[Client, AsyncClient],
    system: str = None,
    save_messages: bool = None,
    params: Dict[str, Any] = None,
) -> Dict[str, Any]:
    # call 1: select tool and populate context
    tools_list = "\n".join(f"{i+1}: {f.__doc__}" for i, f in enumerate(tools))
    tool_prompt_format = TOOL_PROMPT.format(tools=tools_list)

    logit_bias_weight = 100
    logit_bias = {str(k): logit_bias_weight for k in range(15, 15 + len(tools) + 1)}

    tool_idx = int(
        chat_gpt_session.gen(
            prompt,
            client=client,
            system=tool_prompt_format,
            save_messages=False,
            params={
                "temperature": 0.0,
                "max_tokens": 1,
                "logit_bias": logit_bias,
            },
        )
    )

    # if no tool is selected, do a standard generation instead.
    if tool_idx == 0:
        return {
            "response": chat_gpt_session.gen(
                prompt,
                client=client,
                system=system,
                save_messages=save_messages,
                params=params,
            ),
            "tool": None,
        }
    selected_tool = tools[tool_idx - 1]
    context_dict = selected_tool(prompt)
    if isinstance(context_dict, str):
        context_dict = {"context": context_dict}

    context_dict["tool"] = selected_tool.__name__

    # call 2: generate from the context
    new_system = f"{system or chat_gpt_session.system}\n\nYou MUST use information from the context in your response."
    new_prompt = f"Context: {context_dict['context']}\n\nUser: {prompt}"

    context_dict["response"] = chat_gpt_session.gen(
        new_prompt,
        client=client,
        system=new_system,
        save_messages=False,
        params=params,
    )

    # manually append the nonmodified user message + normal AI response
    user_message = ChatMessage(role="user", content=prompt)
    assistant_message = ChatMessage(role="assistant", content=context_dict["response"])
    chat_gpt_session.add_messages(user_message, assistant_message, save_messages)

    return context_dict